# Run models on Ollama
The purpose of this notebook is to explore running models using my ollama server.

In [1]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [2]:
llm = Ollama(base_url="http://192.168.86.2:11434", model="llama3")


In [3]:
print(llm.invoke("Tell me a joke."))

Here's one:

Why don't eggs tell jokes?

Because they'd crack each other up!

Hope that made you smile!


Try the alternate libraries.

In [4]:
from langchain_community.llms import Ollama

llm = Ollama(base_url="http://192.168.86.2:11434", model="llama3")
print(llm("Why is the sky blue?"))


/opt/conda/envs/phi3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


What a great question!

The sky appears blue to our eyes because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh (1842-1919). Here's what happens:

1. **Sunlight**: When sunlight enters Earth's atmosphere, it consists of all the colors of the visible spectrum, including red, orange, yellow, green, blue, indigo, and violet.
2. **Molecules**: The atmosphere is made up of tiny molecules of gases like nitrogen (N2) and oxygen (O2), which are much smaller than the wavelength of light.
3. **Scattering**: When sunlight hits these molecules, it scatters in all directions. This scattering occurs more frequently for shorter wavelengths of light, like blue and violet, than for longer wavelengths, like red and orange.

As a result of this scattering, the shorter wavelengths of light (like blue and violet) are distributed throughout the sky, making it appear blue to our eyes. The other colors, like red and orange, are not scattered as much and continue to

In [5]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["topic"], 
    template="Give me 5 interesting facts about {topic}."
)
chain = LLMChain(llm=llm, prompt=prompt)

print(chain.run("The space time continuum"))

/opt/conda/envs/phi3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/opt/conda/envs/phi3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The fabric of reality itself! Here are five interesting facts about the space-time continuum:

1. **Time is relative**: According to Einstein's theory of special relativity, time is not absolute. It depends on the observer's frame of reference and their speed relative to other objects in motion. This means that two observers moving at different speeds can experience time differently, with the faster-moving observer aging less than the slower-moving one.
2. **The universe has four dimensions**: In addition to the three spatial dimensions (length, width, and height) we're familiar with, our universe also has a fourth dimension: time. This is often referred to as spacetime. The concept of spacetime was first proposed by Hermann Minkowski in 1907 and later developed by Einstein.
3. **Wormholes are theoretical shortcuts**: Wormholes are hypothetical tunnels or tubes that could connect two distant points in space-time, allowing for faster-than-light travel. They would effectively create a sh

OK. Now moving on lets investigate using llama 3 to do function calls with langchain.

In [4]:
from langchain.chat_models import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOllama(base_url="http://192.168.86.2:11434", model="llama3", keep_alive=-1, temperature=0.0, max_new_tokens=512)


In [5]:
prompt = ChatPromptTemplate.from_template("Write a 500 word article omn the {topic} from the perspective of a {profession}.")
#chain = prompt | llm | StrOutputParser()
chain = prompt | llm
print(chain.invoke({"topic": "astronomy", "profession": "engineer"}))

content="As an engineer, I have always been fascinated by the wonders of the universe and the incredible feats of engineering that allow us to explore and understand it. Astronomy is not just about gazing up at the stars; it's about applying scientific principles and technological innovations to unravel the mysteries of the cosmos.\n\nOne of the most impressive examples of astronomical engineering is the Hubble Space Telescope. Launched in 1990, this orbiting observatory has revolutionized our understanding of the universe by capturing stunning images and gathering valuable data on distant galaxies, stars, and planets. The Hubble's 2.4-meter primary mirror and advanced instruments allow it to peer into the farthest reaches of space, revealing hidden secrets about the formation and evolution of the cosmos.\n\nAnother remarkable example is the Atacama Large Millimeter/submillimeter Array (ALMA) in Chile. This radio telescope array consists of 66 antennas working together to form a single

In [12]:
print(prompt)

input_variables=['profession', 'topic'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['profession', 'topic'], template='Write a 500 word article omn the {topic} from the perspective of a {profession}.'))]


In [7]:
for chunk in chain.stream({"topic": "astronomy", "profession": "engineer"}):
    print(chunk, end="", flush=True)

As an engineer, I have always been fascinated by the wonders of the universe and the incredible feats of engineering that go into understanding it. Astronomy is not just about gazing up at the stars; it's about applying scientific principles to unravel the mysteries of the cosmos.

One of the most impressive aspects of astronomy is the sheer scale of the projects involved. Take, for example, the construction of a telescope like the Mauna Kea Observatory in Hawaii. This behemoth of a telescope stands over 13,000 feet above sea level and has a primary mirror spanning 8.2 meters in diameter. To put that into perspective, it's roughly the size of a small swimming pool! The engineering required to design and build such a massive structure is staggering.

But it's not just about building big; astronomy also requires precision and attention to detail. Take, for instance, the process of designing and launching a spacecraft like NASA's Kepler mission. This tiny probe was tasked with monitoring 

In [8]:
from langchain_experimental.llms.ollama_functions import OllamaFunctions

llm = OllamaFunctions(base_url="http://192.168.86.2:11434", model="llama3", format="json")

model = llm.bind_tools(
    tools=[
        {
            "name": "get_current_weather",
            "description": "get the current weather in a given location",
            "parameters": {
                "type": "object", 
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "default": "celsius",
                    },
                },
                "required": ["location"],
            },
        }
    ],
    function_call={"name": "get_current_weather"},
)


response = model.invoke("What is the weather in Sydney?")

print(response)

content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': '{"location": "Sydney, Australia", "unit": "celsius"}'}} id='run-419101c9-847b-451e-9cb3-58303ea37225-0'


In [5]:
response = model.invoke("Tell me a joke about bananas.")

print(response)

content='' additional_kwargs={'function_call': {'name': 'get_current_weather', 'arguments': ''}} id='run-4cd9eaf2-2955-46f8-a9f8-69a0252b4fc8-0'


Now lets implement an agent. See if we can get it running on ollama.

In [9]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()
response = search("What is the capital of France?")

print(response)

[{'url': 'https://www.britannica.com/place/France', 'content': 'Even though its imperialist stage was driven by the impulse to civilize that world according to French standards (la mission civilisatrice), the French still note approvingly the words of writer Gustave Flaubert:\nI am no more modern than I am ancient, no more French than Chinese; and the idea of la patrie, the fatherland—that is, the obligation to live on a bit of earth coloured red or blue on a map, and to detest the other bits coloured green or black—has always seemed to me narrow, restricted, and ferociously stupid.\n Among France’s other major cities are Lyon, located along an ancient Rhône valley trade route linking the North Sea and the Mediterranean; Marseille, a multiethnic port on the Mediterranean founded as an entrepôt for Greek and Carthaginian traders in the 6th century bce; Nantes, an industrial centre and deepwater harbour along the Atlantic coast; and Bordeaux, located in southwestern France along the Garo

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [3]:
retriever.invoke("how to upload a dataset")[0]

Document(page_content='description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";co

In [5]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [2]:
import faiss

print("Faiss version:", faiss.__version__)


Faiss version: 1.8.0


In [8]:
tools = [search, retriever_tool]

Now we need to create a llama3 specific prompt. This will be a learning experience.

In [13]:
prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|> 
You are a helpful assistant. <|eot_id|>
<|start_header_id|>placeholder<|end_header_id|> 
{chat_history}  <|eot_id|>
<|start_header_id|>human<|end_header_id|> 
{input}  <|eot_id|>
<|start_header_id|>placeholder<|end_header_id|> 
{agent_scratchpad}  <|eot_id|>
"""

agent_prompt = ChatPromptTemplate.from_template(prompt)

print(agent_prompt)

input_variables=['agent_scratchpad', 'chat_history', 'input'] messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['agent_scratchpad', 'chat_history', 'input'], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|> \nYou are a helpful assistant. <|eot_id|>\n<|start_header_id|>placeholder<|end_header_id|> \n{chat_history}  <|eot_id|>\n<|start_header_id|>human<|end_header_id|> \n{input}  <|eot_id|>\n<|start_header_id|>placeholder<|end_header_id|> \n{agent_scratchpad}  <|eot_id|>\n'))]
